In [1]:
import sys
sys.path.append('../code')

In [2]:
import json
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
from make_patch_df import *
from patch_nlp import get_info_from_df
from add_draft_info import *

In [4]:
hero_name_set = get_hero_names()
ability_set = get_abi_names('../../dota2-database/json/abilities.json')

In [5]:
patch_url_dict = {}
patch_url_dict['6.86'] = 'December_16,_2015_Patch'
patch_url_dict['6.85'] = 'September_24,_2015_Patch'
patch_url_dict['6.84'] = 'April_30,_2015_Patch'
patch_url_dict['6.83'] = 'December_17,_2014_Patch'
patch_url_dict['6.82'] = 'September_25,_2014_Patch'
patch_url_dict['6.81'] = 'April_29,_2014_Patch'
patch_url_dict['6.80'] = 'January_29,_2014_Patch'
patch_url_dict['6.79'] = 'October_21,_2013_Patch'
patch_url_dict['6.78'] = 'June_04,_2013_Patch'
patch_url_dict['6.77'] = 'December_19,_2012_Patch'

In [6]:
with open('../data/patch_685_df.pkl') as f:
    df_685_full = pickle.load(f)
    
# df_685 = df_from_patch_query('September_24,_2015_Patch', hero_name_set, '6.85')

In [7]:
df_685_full.head(6)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,0,2,"[-1, -1]",[distance rescaled from 1000/1075/1150/1150 to...
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],1,0,[1],[Spark Wraith cooldown reduced from 7 to 4]
5,Axe,6.85,[],0,0,[],[]


In [8]:
texts_685, ratios_685, labels_685 = get_info_from_df(df_685_full, labeled=True)

In [54]:
tfidf_train = TfidfVectorizer(use_idf=False)
tfmat_train_685 = tfidf_train.fit_transform(texts_685)
gdb_685 = GradientBoostingClassifier(random_state=0)
gdb_685.fit(tfmat_train_685.toarray(), labels_685)

all_patches = get_patch_dfs(patch_url_dict, hero_name_set, ability_set, tfidf_train, gdb_685)

In [10]:
all_patches['6.83'].head(3)

,hero,patch,text,text_no_abi,predictions,ratios
0,Abaddon,6.83,[Borrowed Time duration increased from 3/4/5 t...,[duration increased from 3/4/5 to 4/5/6],[0.9509],[1.25]
1,Alchemist,6.83,[Greevil's Greed:\n\n Base bonus gold increase...,[:\n\n Base bonus gold increased from 4/6/8/10...,[0.9158],[1.286]
2,Ancient Apparition,6.83,[],[],[],[]


In [11]:
df_685_full.head(3)

,hero,patch,text,num_buffs,num_nerfs,change_type,text_no_abi
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...",2,0,"[1, 1]","[cooldown reduced from 5 to 4.5, Aghanim's Sce..."
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,2,0,"[1, 1]",[damage increased from 12/16/20/24 to 15/20/25...
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,1,0,[1],[cast range increased from 700 to 700/800/900/...


In [12]:
df_685 = df_685_full.copy()

In [13]:
df_685.drop(['num_buffs', 'num_nerfs'], axis=1, inplace=True)

In [14]:
counter = 0
ratios_by_hero = []
for item in df_685['text']:
    ratios_by_hero.append(ratios_685[counter:counter+len(item)])
    counter += len(item)
df_685['ratios'] = ratios_by_hero

In [15]:
ratios_685[136]

inf

In [16]:
counter = 0
for item in df_685['text']:   
    counter += len(item)
    if counter > 136:
        print item
        break

[u'Agility growth reduced from 3 to 2.6', u'Doppelganger cast time increased from 0 to 0.1']


In [17]:
# Working as intended, with inf resulting from 0.1 / 0 division. Still need to address.

In [18]:
df_685.rename(columns={'change_type':'predictions'}, inplace=True)
df_685 = df_685[list(df_685.columns[:3]) + ['text_no_abi', 'predictions', 'ratios']]
df_685.head()

,hero,patch,text,text_no_abi,predictions,ratios
0,Abaddon,6.85,"[Mist Coil cooldown reduced from 5 to 4.5, Bor...","[cooldown reduced from 5 to 4.5, Aghanim's Sce...","[1, 1]","[0.9, 1.429]"
1,Alchemist,6.85,[Acid Spray damage increased from 12/16/20/24 ...,[damage increased from 12/16/20/24 to 15/20/25...,"[1, 1]","[1.25, 1.25]"
2,Ancient Apparition,6.85,[Cold Feet cast range increased from 700 to 70...,[cast range increased from 700 to 700/800/900/...,[1],[1.214]
3,Anti-Mage,6.85,[Blink distance rescaled from 1000/1075/1150/1...,[distance rescaled from 1000/1075/1150/1150 to...,"[-1, -1]","[0.949, 0.75]"
4,Arc Warden,6.85,[Spark Wraith cooldown reduced from 7 to 4],[Spark Wraith cooldown reduced from 7 to 4],[1],[0.571]


In [ ]:
all_patches['6.85'] = df_685

In [63]:
all_drafts = get_draft_dfs('../data/draft_stats/')
all_drafts = prepare_draft_dfs(all_drafts)
all_drafts[677].head(3)

In [31]:
[int(float(key)*100) for key in all_patches.iterkeys()]

[680, 681, 682, 683, 684, 678, 686, 677, 679]

In [32]:
[str(key/100.) for key in all_drafts.iterkeys()]

['6.77', '6.78', '6.79', '6.8', '6.81', '6.82', '6.83', '6.84', '6.85', '6.86']

In [76]:
all_patches['6.77'].merge(all_drafts[677], how='right', on='hero').head(3)

,hero,patch,text,text_no_abi,predictions,ratios,pick%,ban%,pb%,times_pb,prev_patch_pb%
0,Alchemist,6.77,[Unstable Concoction:\n\n Fixed charge time co...,[:\n\n Fixed charge time counting when it was ...,[0.5401],[None],9.4,10.1,19.5,338,NaN
1,Ancient Apparition,6.77,[Chilling Touch manacost rescaled from 140 to ...,[manacost rescaled from 140 to 110/120/130/140],[0.4682],[0.893],2.7,1.3,4.0,69,NaN
2,Anti-Mage,6.77,[],[],[],[],15.2,18.9,34.1,592,NaN


In [ ]:
# notice AA buff is painfully misclassified

In [66]:
all_patches['6.84'].merge(all_drafts[684], how='right', on='hero')

,hero,patch,text,text_no_abi,predictions,ratios,pick%,ban%,pb%,times_pb,prev_patch_pb%
0,Abaddon,6.84,[Borrowed Time can now be cast while disabled ...,[can now be cast while disabled (same rules as...,[0.9248],[None],1.6,0.6,2.2,34,4.9
1,Alchemist,6.84,[Unstable Concoction:\n\n Countdown can now be...,[:\n\n Countdown can now be seen by enemies\n\...,"[0.8011, 0.7333, 0.942]","[None, None, None]",1.2,0.3,1.4,22,1.1
2,Ancient Apparition,6.84,[Chilling Touch buff can now be removed by cli...,[buff can now be removed by clicking on the bu...,[0.6352],[None],9.9,15.1,25.0,391,44.3
3,Anti-Mage,6.84,[],[],[],[],9.5,14.7,24.2,378,17.0
4,Axe,6.84,"[Berserker's Call cast point from 0.3 to 0.4, ...","[cast point from 0.3 to 0.4, now triggers when...","[0.9509, 0.8303, 0.6542]","[1.333, None, 0.75]",5.5,5.3,10.8,169,72.6
5,Bane,6.84,[Brain Sap mana cost from 125/150/175/200 to 1...,[mana cost from 125/150/175/200 to 100/125/150...,"[0.8633, 0.2249, 0.8303]","[0.846, None, None]",8.6,3.1,11.7,183,4.0
6,Batrider,6.84,[Flamebreak damage changed from an instant 75/...,[damage changed from an instant 75/150/225/300...,"[0.6433, 0.1837]","[0.267, 0.8]",2.0,3.6,5.6,88,77.1
7,Beastmaster,6.84,"[Base damage increased by 4, Wild Axes cast po...","[Base damage increased by 4, cast point from 0...","[0.9565, 0.8261, 0.6542]","[None, 0.8, 0.79]",11.9,13.7,25.6,400,15.9
8,Bloodseeker,6.84,[Thirst:\n\n Bonuses scale from 100 to 25% HP ...,[:\n\n Bonuses scale from 100 to 25% HP instea...,[0.1144],[0.25],9.0,14.1,23.1,361,3.1
9,Bounty Hunter,6.84,[Intelligence growth increased from 1.4 to 2.0...,[Intelligence growth increased from 1.4 to 2.0...,"[0.9509, 0.9093, 0.5173]","[1.429, 1.2, None]",17.2,34.1,51.3,801,1.0


In [68]:
all_patches['6.84'].ix[0,:]['text']

[u"Borrowed Time can now be cast while disabled (same rules as Morphling's Morph)"]

In [70]:
patch_draft_dfs = {}
for key, patch_df in all_patches.iteritems():
    key_as_int = int(float(key)*100)
    merged_df = patch_df.merge(all_drafts[key_as_int], how='right', on='hero')
    patch_draft_dfs[key] = merged_df

In [75]:
patch_draft_dfs['6.78']

,hero,patch,text,text_no_abi,predictions,ratios,pick%,ban%,pb%,times_pb,prev_patch_pb%
0,Abaddon,6.78,[],[],[],[],7.5,10.3,17.8,399,NaN
1,Alchemist,6.78,[],[],[],[],26.9,30.2,57.1,1280,19.5
2,Ancient Apparition,6.78,"[Ice Vortex cooldown decreased from 6 to 5., I...","[cooldown decreased from 6 to 5., :\n\n Base A...","[0.9356, 0.9509]","[0.833, 1.222]",2.0,0.7,2.7,60,4.0
3,Anti-Mage,6.78,[],[],[],[],12.2,15.7,27.9,626,34.1
4,Axe,6.78,[Counter Helix cooldown decreased from 0.65/0....,[cooldown decreased from 0.65/0.6/0.55/0.5 to ...,[0.9356],[0.826],0.6,0.2,0.8,18,1.1
5,Bane,6.78,[],[],[],[],19.0,14.9,33.9,760,40.1
6,Batrider,6.78,"[Vision decreased from 1800/800 to 1400/800., ...","[Vision decreased from 1800/800 to 1400/800., ...","[0.1163, 0.9453]","[0.846, 1.273]",21.1,78.7,99.7,2236,99.9
7,Beastmaster,6.78,[Call of the Wild's Greater Hawk invisibility ...,['s Greater Hawk invisibility fade time decrea...,[0.5421],[0.8],8.2,3.2,11.3,254,15.5
8,Bloodseeker,6.78,[Thirst now gives you bonus 3/4/5/6 armor when...,[now gives you bonus 3/4/5/6 armor when active...,"[0.8303, 0.8303, 0.8477]","[None, None, None]",0.5,0.2,0.8,17,0.1
9,Bounty Hunter,6.78,[Track manacost from 70/60/50 to 50.],[manacost from 70/60/50 to 50.],[0.7742],[0.833],12.8,8.5,21.4,479,20.4
